<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto (Canada)</font></h1>

## 1. Create a dataframe with Toronto data

First import the libraries that will be needed.

In [49]:
import pandas as pd
import numpy as np

!conda install -c conda-forge wikipedia --yes
import wikipedia as wp


Solving environment: - ^C
failed

CondaError: KeyboardInterrupt



Get Toronto data from Wikipedia page.

In [102]:
html = wp.page("List_of_postal_codes_of_Canada:_M").html().encode("UTF-8")
df = pd.read_html(html)[0]
df = df[df['Borough'] != 'Not assigned']   # Ignore cells with a borough that is Not assigned.
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


Combine neighbourhoods in the same Postal Code Areas into one row with the neighborhoods separated with a comma.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
Use the .shape method to print the number of rows of your dataframe.

In [110]:
columnnames = df.columns
df_new = pd.DataFrame(columns=columnnames)
Postcode = df['Postcode'].unique()

for pc in Postcode:
    pos = df['Postcode'] == pc
    borough = df['Borough'][pos].unique()
    neighbourhoods = df['Neighbourhood'][pos]
    neighbourhoods.mask(neighbourhoods=='Not assigned',borough, inplace = True)
    neighbourhoods = neighbourhoods.str.cat(sep=',')
    df_new = df_new.append({'Postcode':pc, 'Borough':np.array2string(borough), 'Neighbourhood':neighbourhoods}, ignore_index=True)
    
print('The dataframe has {} rows.'.format(df_new.shape[0]))
df_new.head()

The dataframe has 103 rows.


,Postcode,Borough,Neighbourhood
0,M3A,['North York'],Parkwoods
1,M4A,['North York'],Victoria Village
2,M5A,['Downtown Toronto'],Harbourfront
3,M6A,['North York'],"Lawrence Heights,Lawrence Manor"
4,M7A,['Downtown Toronto'],Queen's Park


## 2. Get the latitude and the longitude coordinates of each neighborhood

Install and imprt Geocoder library

In [134]:
!conda install -c conda-forge geocoder --yes
import geocoder # import geocoder

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

    geocoder: 1.38.1-py_1 conda-forge
    ratelim:  0.1.6-py_2  conda-forge


geocoder-1.38.1      | 53 KB     | ##################################### | 100% 
ratelim-0.1.6        | 6 KB      | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [173]:
# initialize variables
Latitude=[]
Longitude=[]

for pc in Postcode:
    # loop until you get the coordinates
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(pc))
        lat_lng_coords = g.latlng
    Latitude.append(lat_lng_coords[0])
    Longitude.append(lat_lng_coords[1])


In [181]:
# Add the latitude and longitude data of e
df_new['Latitude'] = Latitude
df_new['Longitude'] = Longitude

df_new.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,['North York'],Parkwoods,43.752420,-79.329242
1,M4A,['North York'],Victoria Village,43.730600,-79.313265
2,M5A,['Downtown Toronto'],Harbourfront,43.650295,-79.359166
3,M6A,['North York'],"Lawrence Heights,Lawrence Manor",43.723270,-79.451286
4,M7A,['Downtown Toronto'],Queen's Park,43.661150,-79.391715


## 3. Explore and cluster the neighborhoods in Toronto